In [1]:
"""
Uninstall current version of polyscope from current virtual environment:
    pip uninstall polyscope

Local installation. Haven't put this version on PyPI yet, since it seems like uploading a 
new version could take over an hour. 
  git clone git@github.com:i-geng/polyscope-py.git

From the root polyscope-py directory, need to build the C++ source:
    mkdir build; cd build
    cmake ../
    make

Back in the root polyscope-py directory:
    pip install .
"""

import numpy as np
import polyscope as ps
import potpourri3d as pp3d

In [5]:
""" Polyscope set-up """
ps.set_ground_plane_mode("none")
ps.set_background_color(np.array([0.0, 0.0, 0.0]))
ps.init()

[polyscope] Backend: openGL3_glfw -- Loaded openGL version: 4.1 INTEL-22.5.11


In [3]:
""" Helper functions """
def load_surface_mesh(filename: str, obj_name: str):
    verts, faces = pp3d.read_mesh(filename)
    return ps.register_surface_mesh(obj_name, verts, faces, enabled=True)

def polarToCartesian(r, theta, phi):
    theta = np.deg2rad(theta)
    phi = np.deg2rad(phi)  
    x = r * np.sin(theta) * np.cos(phi)
    y = r * np.sin(theta) * np.sin(phi)
    z = r * np.cos(theta)
    return np.array([x, y, z])

In [9]:
""" Write video file """
def write_video_file_demo(filename):
    r, phi = 8, 20
    frames = 120
    theta_range = [0, 360]

    # Open a video file, which returns a file descriptor (in a python capsule)
    fd = ps.open_video_file(filename, fps=30)
    
    for i in range(frames):
        theta = theta_range[0] + (theta_range[1] - theta_range[0]) * i / frames
        cam_loc = polarToCartesian(r, theta, phi) + np.array([0, 4, 0])
        ps.look_at(cam_loc, [0, 2, 0])
        
        # Call ps.write_video_frame(fd) instead of ps.screenshot()
        ps.write_video_frame(fd, transparent_bg=True)

    # Close the video file
    ps.close_video_file(fd)

    
""" Point light set-up """
def point_light_demo():
    # Registering multiple point lights
    # As of now, can register up to 10 point lights
    light1_pos = np.array([0, 5, 5])
    light1_col = np.array([0.0, 0.0, 1.0])
    light1 = ps.register_point_light("light 1", light1_pos, light1_col)

    light2_pos = np.array([0, 5, -5])
    light2_col = np.array([1.0, 0.0, 0.0])
    light2 = ps.register_point_light("light 2", light2_pos, light2_col)

    light3_pos = np.array([0, 3, 0])
    light3_col = np.array([1.0, 1.0, 1.0])
    light3 = ps.register_point_light("light 3", light3_pos, light3_col)

    light4_pos = np.array([0, -1, 0])
    light4_col = np.array([0.5, 0.5, 0.5])
    light4 = ps.register_point_light("light 4", light4_pos, light4_col)

    # Other point light functions:
    light1.set_enabled(False)
    light1.set_position(np.array([0, 5, 6]))
    light1.set_color(np.array([0.0, 0.1, 1.0]))
    light1.set_enabled(True)


In [10]:
""" Demo 1 """
ps_mesh = load_surface_mesh("./data/teapot.obj", "teapot")
ps_mesh.set_material("phong")
ps_mesh.set_color(np.array([1.0, 1.0, 1.0]))

point_light_demo()
write_video_file_demo("./out/teapot.mp4")

AttributeError: 'PointLight' object has no attribute 'set_enabled'

: 

In [7]:
""" Demo 2 """
ps_mesh = load_surface_mesh("./data/teapot.obj", "teapot")
ps_mesh.set_material("flat_tetra")

# Generate random tetracolors
num_faces = ps_mesh.n_faces()
face_colors = np.random.rand(num_faces, 3)
q_values = 0.8 + 0.2 * np.random.rand(num_faces, 1)
face_tetracolors = np.hstack((face_colors, q_values))

# Add a tetracolor to each face
ps_mesh.add_tetracolor_quantity("face tetracolor", face_tetracolors, defined_on='faces', enabled=True)

# Must set material to "flat_tetra" to view a tetracolor quantity
ps_mesh.set_material("flat_tetra")

# Rasterize a tetracolor image
ps.rasterize_tetra("./out/tetra_teapot.png")

ps.show()